# Исследование надёжности заёмщиков

В наличии база кредитного отдела банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

**Описание данных**

- `children` - количество детей у клиента
- `days_employed` - трудовой стаж в днях
- `dob_years` - возраст клиента
- `education` - уровень образования клиента
- `education_id` - идентификатор уровня образования
- `family_status` - семейное положение
- `family_status_id` - идентификатор семейного положения
- `gender` - пол
- `income_type` - тип занятости
- `debt` - имел ли задолженность по возврату кредитов
- `total_income` - ежемесячный доход
- `purpose` - цель получения кредита

В ходе исследования планируется проанализировать данные банка на предмет пропусков, аномальных значений, ошибок заполнения. Затем данные будут предобработаны на основе выявленных проблем, а также сгруппированы для дальнейшего анализа. Наконец, по подготовленной базе будут построены сводные таблицы, которые помогут определить, какие параметры в большей степени влияют на возврат или невозврат клиентом кредита.

**Структура работы:**


1. [Загрузка данных и первый взгляд](#data_import)
2. [Предобработка данных](#preprocessing)
    * [Обработка пропусков](#missing)
    * [Замена типа данных](#types)
    * [Обработка дубликатов](#duplicates)
    * [Лемматизация](#lemmas)
    * [Категоризация данных](#categories)
3. [Ответы на вопросы. Зависимость возврата кредита в срок от различных параметров](#answers)
    * [Наличие детей](#children)
    * [Семейное положение](#family_status)
    * [Уровень дохода](#income)
    * [Цель кредита](#purpose)
4. [Общий вывод](#conclusion)

<a id="data_import"></a>
### Шаг 1. Загрузка данных и первый взгляд

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from collections import Counter

In [2]:
df = pd.read_csv('/datasets/data.csv')
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

В базе имеем **21 525** строк и **12** переменных. Чтобы не запутаться, проясним сразу, что обозначает каждая переменная, а также дадим краткий комментарий при необходимости:
- `children` - количество детей у клиента, *программа считала переменную корректно как целочисленную, поэтому на данном этапе замена типа не потребуется*
- `days_employed` - трудовой стаж в днях *здесь с ходу видим две проблемы*:
        a. встречаются отрицательные значения переменных; предположительно это связано с тем, что перепутаны
        начальная и конечная даты работы
        b. в данной переменной встречаются пропущенные значения, что, скорее всего, связано с тем, что у клиента
        отсутствует трудовой стаж
    
- `dob_years` - возраст клиента, *тип переменной считан корректно - целочисленный*
- `education` - уровень образования клиента, *видны проблемы с регистром*
- `education_id` - идентификатор уровня образования
- `family_status` - семейное положение
- `family_status_id` - идентификатор семейного положения
- `gender` - пол
- `income_type` - тип занятости, *значения в данном столбце надо будет изучить, на данном этапе не до конца ясно, как именно проведена классификация и, соответственно, какую информацию из этого можно будет извлечь*
- `debt` - имел ли задолженность по возврату кредитов
- `total_income` - ежемесячный доход, *в данной переменной так же, как и в переменной `days_employed` встречаются пропущенные значения, что подтверждает предположение о том, что пропуски означают отсутствие трудового стажа и текущей работы у клиента*
- `purpose` - цель получения кредита

<a id="preprocessing"></a>
### Шаг 2. Предобработка данных

<a id="missing"></a>
### Обработка пропусков

In [3]:
df[df['days_employed'].isnull()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Предположение о том, что значения в переменных со стажем и доходом пропущены у безработных, не подтвердились. В таком случае нужно заполнить эти данные, поскольку они составляют около **10%** нашей выборки. 

В графе со стажем у нас встречаются отрицательные значения. Вероятно, при расчете стажа были перепутаны местами даты начала и окончания трудового стажа либо отрицательные значения используются в случае, когда кредит уже взят и отражают стаж до оформления кредита.

In [4]:
df['days_employed'] = df['days_employed'].abs()
df['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

Как видно из описательной статистики значений трудового стажа, в данных присутствует ошибка, в результате которой стандартное отклонение величины стажа превышает продолжительность человеческой жизни. Вероятно, часть значений указана не в днях, а в часах. Для ответа на вопросы кредитного отдела данные столбца о трудовом стаже не потребуются, поэтому заполнять пропущенные значения в данном случае нет необходимости. Перейдем к данным о ежемесячном доходе.

In [5]:
(df['total_income'] / 1000).describe()

count    19351.000000
mean       167.422302
std        102.971566
min         20.667264
25%        103.053153
50%        145.017938
75%        203.435068
max       2265.604029
Name: total_income, dtype: float64

Пропущенные данные о заработной плате заполним медианой внутри типа занятости.

In [6]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))

In [7]:
df['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [8]:
len(df[df['dob_years'] == 0])

101

В переменной возраст 101 строка с нулевым значением, которые можно трактовать как пропущенные. Однако для ответа на поставленный вопрос данный столбец не понадобится, поэтому пропуски можно не заполнять.

### Вывод

 - Выявлены пропущенные значения в двух столбцах: `days_employed`, `total_income`.
 - В `days_employed` обнаружены отрицательные значения, после исправления этих значений оказалось, что присутствуют слишком большие значения. Принято решение не заполнять пропущенные значения, так как нет уверенности в природе ненормальных значений, а данная переменная не является необходимой для ответа на поставленные вопросы.
 - Пропуски в данных о заработной плате заполнены медианным значением соответствующего типа занятости.

<a id="types"></a>
### Замена типа данных

Практически все переменные в данных считались корректно. Для удобства переведем данные поле в формат целочисленных значений, где `1 = М`, а `0 = F`. Избавимся от столбца с трудовым стажем. Заработную плату превратим в целочисленные значения, поскольку значения с точностью до рубля нас вполне устраивают, а расчеты целочисленных значений будут производиться значительно быстрее.

In [9]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В базе оказалось одно некорректное значение пола. Можем исключить эту строку.

In [10]:
df = df[df['gender'] != 'XNA']

In [11]:
df['gender'] = df['gender'].map({'M' : 1, 'F' : 0})

In [12]:
df['total_income'] = df['total_income'].astype('int')

### Вывод

- значения `gender` переведены из строковых в целочисленные, где **1** - мужской пол, а **0** - женский
- `total_income` переведен в целочисленный формат

<a id="duplicates"></a>
### Обработка дубликатов

Проанализируем категориальные переменные `education`, `family_status`, `income_type` на предмет наличия дубликатов.

In [13]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      667
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Есть проблемы с регистром, вероятно, это связано с ручным вводом данных. Исправим методом `.lower`.

In [14]:
df['education'] = df['education'].str.lower()

In [15]:
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

In [16]:
df['education_id'].value_counts()

1    15233
0     5260
2      743
3      282
4        6
Name: education_id, dtype: int64

Теперь все в порядке. Количество категорий образования совпадает с количеством идентификаторов. Посмотрим на семейное положение.

In [17]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4176
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [18]:
df['family_status_id'].value_counts()

0    12380
1     4176
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Здесь дубликаты отсутствуют. Для единообразия приведем все значения к нижнему регистру.

In [19]:
df['family_status'] = df['family_status'].str.lower()

Наконец разберемся с типом занятости.

In [20]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

В этой переменной дубликатов так же не выявлено. Посмотрим на общее количество дубликатов.

In [21]:
df.duplicated().sum()

71

In [22]:
len(df)

21524

Всего дубликатов **71**. Это связано с тем, что в базе отсутствует переменная с id клиента, а совпадение данных по всем остальным переменным возможно. Тем более, часть данных о заработной плате были заменены на медианное значение. Учитывая, что всего в базе более 20 тыс. значений, дубликаты составляют менее процента, поэтому их можно удалить, не опасаясь искажения результата.

In [23]:
df = df.drop_duplicates().dropna()

In [24]:
df.duplicated().sum()

0

### Вывод

- Произведен поиск дубликатов в трех переменных: `education`, `family_status`, `income_type`
- обнаружены дубликаты в `education`, после приведения значений к нижнему регистру дубликатов не осталось
- в `family_status` и `income_type` дубликаты отсутствуют
- в `family_status` значения для единообразия приведены к нижнему регистру
- во всей базе оказался 71 полный дубликат, поскольку это менее 1% от всех наблюдений, принято решение исключить их

<a id="lemmas"></a>
### Лемматизация

Упакуем все уникальные цели получения кредита в отдельный список. Выделим леммы в этом списке и выведем их в порядке убывания. 

In [25]:
purpose = list(df['purpose'].unique())

In [26]:
lemmas = []
m = Mystem()
lemmas = []
for i in purpose:
    lemmas.extend(m.lemmatize(i))

In [27]:
print(purpose)
print(len(purpose))

['покупка жилья', 'приобретение автомобиля', 'дополнительное образование', 'сыграть свадьбу', 'операции с жильем', 'образование', 'на проведение свадьбы', 'покупка жилья для семьи', 'покупка недвижимости', 'покупка коммерческой недвижимости', 'покупка жилой недвижимости', 'строительство собственной недвижимости', 'недвижимость', 'строительство недвижимости', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'операции с коммерческой недвижимостью', 'жилье', 'операции со своей недвижимостью', 'автомобили', 'заняться образованием', 'получение образования', 'сделка с подержанным автомобилем', 'автомобиль', 'свадьба', 'получение дополнительного образования', 'покупка своего жилья', 'операции с недвижимостью', 'получение высшего образования', 'свой автомобиль', 'сделка с автомобилем', 'профильное образование', 'высшее образование', 'покупка жилья для сдачи', 'на покупку автомобиля', 'строительство жилой недвижимости', 'ремонт жилью', 'заняться высшим образованием']
38


In [28]:
print(Counter(lemmas).most_common())

[(' ', 59), ('\n', 38), ('покупка', 10), ('недвижимость', 10), ('автомобиль', 9), ('образование', 9), ('жилье', 7), ('с', 5), ('операция', 4), ('на', 4), ('свой', 4), ('свадьба', 3), ('строительство', 3), ('получение', 3), ('высокий', 3), ('дополнительный', 2), ('для', 2), ('коммерческий', 2), ('жилой', 2), ('заниматься', 2), ('сделка', 2), ('приобретение', 1), ('сыграть', 1), ('проведение', 1), ('семья', 1), ('собственный', 1), ('подержать', 1), ('со', 1), ('подержанный', 1), ('профильный', 1), ('сдача', 1), ('ремонт', 1)]


Можно выделить следующие группы целей получения кредита: покупка или строительство недвижимости, покупка автомобиля, получение образования, организация свадьбы, ремонт. Создадим переменную `purpose_group`, в которой будет храниться название группы по целям, а также переменную `purpose_group_id` с идентификатором этой группы.

In [29]:
purpose_group = []
purpose_group_id = []
for i in df['purpose']:
    i_lemmatized = m.lemmatize(i)
    if 'ремонт' in i_lemmatized:
        purpose_group.append('ремонт')
        purpose_group_id.append(1)
    elif 'свадьба' in i_lemmatized:
        purpose_group.append('свадьба')
        purpose_group_id.append(2)
    elif 'образование' in i_lemmatized:
        purpose_group.append('образование')
        purpose_group_id.append(3)
    elif 'автомобиль' in i_lemmatized:
        purpose_group.append('автомобиль')
        purpose_group_id.append(4)
    elif bool({'недвижимость', 'жилье'} & set(i_lemmatized)) == True:
        purpose_group.append('недвижимость')
        purpose_group_id.append(5)

In [30]:
df['purpose_group'] = purpose_group
df['purpose_group_id'] = purpose_group_id

In [31]:
df['purpose_group'].value_counts()

недвижимость    9215
автомобиль      3897
образование     3597
свадьба         2099
ремонт           542
Name: purpose_group, dtype: int64

In [32]:
df['purpose_group_id'].value_counts()

5    9215
4    3897
3    3597
2    2099
1     542
Name: purpose_group_id, dtype: int64

### Вывод

- Выделены леммы в целях получения кредита. На основе лемматизации все цели разделены на 5 групп:
    1. ремонт
    2. свадьба
    3. образование
    4. автомобиль
    5. недвижимость
- Добавлены два столбца `purpose_group` с группой цели и `purpose_group_id` с числовым идентификатором группы

<a id="categories"></a>
### Категоризация данных

После обработки данных в базе стало достаточно много столбцов, поэтому некоторые категории можно сохранить в отдельные таблицы словари, а в основной базе оставить только их идентификаторы.

In [33]:
edu_dic = df[['education', 'education_id']]
edu_dic = edu_dic.drop_duplicates().reset_index(drop=True)
edu_dic

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [34]:
family_dic = df[['family_status', 'family_status_id']]
family_dic = family_dic.drop_duplicates().reset_index(drop=True)
family_dic

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [35]:
purpose_dic = df[['purpose_group','purpose_group_id']]
purpose_dic = purpose_dic.drop_duplicates().reset_index(drop=True)
purpose_dic

,purpose_group,purpose_group_id
0,недвижимость,5
1,автомобиль,4
2,образование,3
3,свадьба,2
4,ремонт,1


In [36]:
df_min = df.drop(['education', 'family_status', 'purpose_group'], axis=1)
df_min.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group_id
0,1,8437.673028,42,0,0,0,сотрудник,0,253875,покупка жилья,5
1,1,4024.803754,36,1,0,0,сотрудник,0,112080,приобретение автомобиля,4
2,0,5623.422610,33,1,0,1,сотрудник,0,145885,покупка жилья,5
3,3,4124.747207,32,1,0,1,сотрудник,0,267628,дополнительное образование,3
4,0,340266.072047,53,1,1,0,пенсионер,0,158616,сыграть свадьбу,2


Переменные с информацией о размере заработной платы и количестве детей можно сгруппировать для дальнейшего сравнения и оценки платежеспособности.

In [37]:
(df['total_income'] / 1000).describe()

count    19350.000000
mean       167.419916
std        102.973897
min         20.667000
25%        103.040500
50%        145.011000
75%        203.423500
max       2265.604000
Name: total_income, dtype: float64

Разобъем размер зароботной платы клиентов по квартилям.

In [38]:
def income_id(income):
    if income < 107620:
        return 1
    if income < 145017:
        return 2
    if income < 195799:
        return 3
    return 4
df['income_id'] = df['total_income'].apply(income_id)

In [39]:
df['children'].value_counts()

 0     12709
 1      4343
 2      1851
 3       294
 20       67
-1        44
 4        34
 5         8
Name: children, dtype: int64

В количестве детей встречается одно отрицательное значение (возьмем его по модулю), а также после 5 детей сразу идет 20. Возможно, количество детей в семье больше 5 составители кодировали числом 20. Разобъем количество детей в семье так же на 4 группы. Группы с семьями без детей, с 1 и 2 детьми значительно больше остальных групп, поэтому семьи, в которых более 2 детей будут объединены в одну группу.

In [40]:
df['children'] = df['children'].abs()

In [41]:
df['children'].value_counts()

0     12709
1      4387
2      1851
3       294
20       67
4        34
5         8
Name: children, dtype: int64

In [42]:
def child_group(children):
    if children < 1:
        return 1
    if children < 2:
        return 2
    if children < 3:
        return 3
    return 4
df['children_id'] = df['children'].apply(child_group)

### Вывод

- Для дальнейшей оценки переменные `total_income` и `children` были разбиты на группы в зависимости от значения переменной:
    1. по размеру заработной платы клиенты разделены на квартили
    2. по количеству детей в семье - на 4 группы: с 0, 1, 2 и более детьми

<a id="answers"></a>
### Шаг 3. Ответы на вопросы. Зависимость возврата кредита в срок от различных параметров

<a id="children"></a>
**Наличие детей**

Для ответа на поставленные вопросы будем смотреть на долю клиентов, не вернувших кредит и количество клиентов по категориям.

In [43]:
df.groupby('children_id')['debt'].agg(['count', 'mean'])

,count,mean
children_id,,
1,12709,0.074908
2,4387,0.093230
3,1851,0.095624
4,403,0.081886


### Вывод

Сильной зависимости между возвратом кредита и наличием детей нет. Меньше всего невозвратных кредитов у клиентов без детей, чуть больше у многодетных клиентов (более 2 детей), самое большое количество невозвратных кредитов у клиентов с 1-2 детьми. Однако разброс составляет всего 1-2%.

Вероятно, клиенты с большим количеством детей вынуждены более тщательно контролировать свой бюджет, поэтому реже допускают просрочку по кредиту в отличие от клиентов с 1-2 детьми.

<a id="family_status"></a>
**Семейное положение**

In [44]:
df.groupby('family_status')['debt'].agg(['count', 'mean'])

,count,mean
family_status,,
в разводе,1083,0.070175
вдовец / вдова,865,0.064740
гражданский брак,3734,0.090787
женат / замужем,11143,0.075922
не женат / не замужем,2525,0.100594


Семейное положение косвенно указывает на возраст клиентов. По таблице выше получается, чем старше клиент, тем меньше вероятность просрочки по кредиту. Чтобы это проверить, можем заполнить пропуски в переменной возраст и вывести сводную таблицу возвратности кредита по возрасту и семейному положению. Пропуски заполним средним значением внутри группы по семейному положению. Если бы данные по стажу были корректны, то предпочтительнее было бы заполнить по нему. Далее разобъем на 4 возрастные группы по квартилям.

In [46]:
df['dob_years'] = df['dob_years'].replace(0, np.NaN)
df['dob_years'] = df['dob_years'].fillna(df.groupby('family_status')['dob_years'].transform('mean'))
df['age_group'] = pd.qcut(df['dob_years'], 4)
pd.pivot_table(df, values='debt', index=['family_status', 'age_group'], aggfunc=['count', 'mean'])

count      mean
                                      debt      debt
family_status         age_group                     
в разводе             (18.999, 33.0]   162  0.111111
                      (33.0, 43.0]     306  0.062092
                      (43.0, 53.0]     314  0.070064
                      (53.0, 75.0]     301  0.056478
вдовец / вдова        (18.999, 33.0]    13  0.076923
                      (33.0, 43.0]      68  0.044118
                      (43.0, 53.0]     179  0.055866
                      (53.0, 75.0]     605  0.069421
гражданский брак      (18.999, 33.0]  1010  0.107921
                      (33.0, 43.0]    1085  0.099539
                      (43.0, 53.0]     860  0.083721
                      (53.0, 75.0]     779  0.064185
женат / замужем       (18.999, 33.0]  2548  0.104396
                      (33.0, 43.0]    3240  0.081790
                      (43.0, 53.0]    2734  0.066935
                      (53.0, 75.0]    2621  0.050362
не женат / не замужем (18.999, 33.0]  1130  0.123894
                      (33.0, 43.0]     549  0.103825
                      (43.0, 53.0]     422  0.087678
                      (53.0, 75.0]     424  0.047170

Гипотеза подтвердилась: вне зависимости от семейного положения молодые люди до 33 лет чаще имеют задолженности по кредитам.

### Вывод

По измерению семейного положения разброс чуть больше, однако явные "группы риска" выявить не представляется возможным. Люди, потерявшие супруга(у) чаще всего возвращают кредит - в 93% случаев, одинокие люди - реже всего (в 90% случаев).

Более глубокое исследование показало, что люди между 19 и 43 годами чаще имеют задолженность по кредиту. Максимальный процент невозврата у клиентов в возрасте между 19 и 33 годами.

<a id="income"></a>
**Уровень дохода**

In [47]:
df.groupby('income_id')['debt'].agg(['count', 'mean'])

,count,mean
income_id,,
1,5363,0.079620
2,4312,0.087430
3,4311,0.089074
4,5364,0.071402


### Вывод

Уровень дохода из всех рассмотренных параметров хуже всего объясняет вероятность возврата кредита. Разброс между максимальным и минимальным значением составляет 1,6%.

<a id="purpose"></a>
**Цель кредита**

In [48]:
pd.pivot_table(df, values='debt', index='purpose_group', aggfunc=['count', 'mean'])

,count,mean
,debt,debt
purpose_group,,
автомобиль,3897,0.094175
недвижимость,9215,0.074227
образование,3597,0.092021
ремонт,542,0.057196
свадьба,2099,0.075274


### Вывод

Цель кредита, напротив, лучше всего помогает понять, насколько высока вероятность, что клиент вернет кредит. Возвратность кредитов на ремонт почти в 2 раза выше по сравнению с возвратностью кредитов на образование или автомобиль.

<a id="conclusion"></a>
### Шаг 4. Общий вывод

В работе рассмотрена база данных клиентов кредитного отдела банка для выявления параметров, влияющих на вероятность погашения кредита в срок. Для анализа данные в базе были обработаны:
* заполнены пропущенные значения в столбцах `dob_years`, `total_income`
* удалены дубликаты из переменной `education`
* с помощью лемматизации выявлены группы кредитных целей
* осуществлена группировка клиентов по переменным `children` и `dob_years`.

Анализ базы данных показал, что количество детей в семье, семейное положение и уровень дохода не позволяют достаточно надежно оценить вероятность возврата кредита. Разброс между максимальными и минимальными показателями возвратности невелик и составляет 1-3%. При этом вне зависимости от группировки менее 10% клиентов имеют задолженности.

Чуть больше о вероятности возврата кредита говорит его цель. Кредиты на ремонт возвращаются чаще остальных, кредиты на автомобили и образование реже остальных - практически в два раза по сравнению с кредитами на ремонт.

Эффективнее всего вероятность просрочки выплаты по кредиту можно оценить по возрасту клиента. "Группа риска" - клиенты в возрасте от 19 до 33 лет, в возрасте от 33 до 42 лет клиенты также чаще имеют задолженности по сравнению с клиентами более старшего возраста.